In [9]:
import xgboost as xgb 
import pandas as pd 

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OrdinalEncoder 

In [6]:
data = pd.read_csv('apple_quality.csv')

data

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,good
1,1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,good
2,2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,bad
3,3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,good
4,4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,good
...,...,...,...,...,...,...,...,...,...
3995,3995,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784,bad
3996,3996,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235,good
3997,3997,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611,bad
3998,3998,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229720,good


In [15]:
data['Quality'].value_counts()

Quality
good    2004
bad     1996
Name: count, dtype: int64

In [16]:
data['Quality'] = (data['Quality'] == 'good').astype(int)

# num_imputer = SimpleImputer(strategy='constant')

# c_transformer = Pipeline(steps=[
#     ('imputer', (SimpleImputer(strategy='most_frequent'))),
#     ('enc', (OrdinalEncoder()))
# ])

# preprocessor = ColumnTransformer(
#     transformers=[
#         # ('num', num_imputer, num_cols),
#         ('cat', c_transformer, ctg_col)
        
#     ]
# )

In [17]:
x = data.drop(columns=['Quality'])
y = data['Quality']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

model = xgb.XGBClassifier(
                     objective='binary:logistic',
                     max_depth=5,
                     n_estimators=100,
                     random_state=42,
                     learning_rate=0.1,
                )

In [19]:
my_pipeline = Pipeline(steps=[
     # ('preprocessor', preprocessor),
     ('model', model)
])

my_pipeline.fit(x_train, y_train)
prediction = my_pipeline.predict(x_test)
accuracy = my_pipeline.score(x_test, y_test)

accuracy

0.88875

In [20]:
report = classification_report(y_test, prediction)

print(report)

              precision    recall  f1-score   support

           0       0.92      0.86      0.89       422
           1       0.86      0.92      0.89       378

    accuracy                           0.89       800
   macro avg       0.89      0.89      0.89       800
weighted avg       0.89      0.89      0.89       800



In [22]:
cv_score = cross_val_score(my_pipeline, x, y, cv=5, scoring='accuracy')

print(cv_score)
print(f'Average accuracy is {cv_score.mean()}')

[0.8425  0.86625 0.87625 0.86375 0.86125]
Average accuracy is 0.8619999999999999
